# Getting Data
- awesomedatabase; google datasets, kaggle....
- API
- scrapping

## Pandas Profiling


In [ ]:
!pip install pandas_profiling==1.2.0
from pandas_profiling import ProfileReport
import pandas as pd
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandas-profiling: filename=pandas_profiling-1.2.0-py2.py3-none-any.whl size=21810 sha256=45d31f2c9959beb327e6613b6bc6858d72d4242cc7196385005ce34dfa829343
  Stored in directory: /root/.cache/pip/wheels/57/be/20/a4afca74c6b28e32352ffdb0b0a921fc5566515eb61e683fcb
Successfully built pandas-profiling
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 2.9.0
    Uninstalling pandas-profiling-2.9.0:
      Successfully uninstalled pandas-profiling-2.9.0


In [ ]:
prof = ProfileReport(df)
prof.to_file('report.html')
files.download('report.html')

## API
### Geopy
https://geopy.readthedocs.io/

In [ ]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from geopy.geocoders import Nominatim
from geopy import distance
locator = Nominatim()
location = locator.geocode("Karimnagar")
locations = [locator.geocode("Ramagundam"),locator.geocode("Srikakulam"), locator.geocode("Bikaner"),locator.geocode("Chikmagalur")]
for i in locations:
  print(distance.distance((location.latitude, location.longitude), (i.latitude, i.longitude)).miles)

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


31.9663617748615
314.21844048127787
755.647142375346
416.7549290251937


In [ ]:
def di(a,b):
  x = locator.geocode(a)
  y = locator.geocode(b)
  print(distance.distance((x.latitude, x.longitude), (y.latitude, y.longitude)).km)

di("Kakinada","Pithapur")
di("Bhuj","Tiruppur")
di("Kollam","Tirap")
di("Seoni","Damoh")

389.78191088025056
1571.8101755773625
2827.3944846636973
163.85961955995154


## Requests

In [ ]:
import requests
from urllib.parse import urlencode
def location_code(city):
    location_url = "https://locator-service.api.bbci.co.uk/locations/?" + urlencode({
    'api_key': "AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv",
    'stack': "aws",
    'locale': "en",
    'filter': "international",
    'place-types': "settlement,airport,district",
    'order' : "importance",
    's': city,
    'a': "true",
    'format': "json"
    })

    response = requests.get(location_url).json()
    return response['response']['results']['results'][0]['id']

In [ ]:
location_code('Bangalore')

'1277333'

## BS4

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [ ]:
url = 'https://www.imdb.com/chart/top/'
soup = bs(requests.get(url).content)
contents = soup.prettify()
imdb = soup.find(class_="chart full-width")
movie_titles = [row.a.text.strip() for row in imdb.find_all(class_='titleColumn')]
movie_ratings = [row.text.strip() for row in imdb.find_all(class_='ratingColumn imdbRating')]
movie_years = [row.span.text.strip('()') for row in imdb.find_all(class_='titleColumn')]
movies = pd.DataFrame({'movie_title':movie_titles, 'movie_ratings':movie_ratings, 'movie_year': movie_years})
movies.head(5)


,movie_title,movie_ratings,movie_year
0,The Shawshank Redemption,9.2,1994
1,The Godfather,9.2,1972
2,The Dark Knight,9.0,2008
3,The Godfather Part II,9.0,1974
4,12 Angry Men,8.9,1957


In [ ]:
import json                   # to convert API to json format

from urllib.parse import urlencode

import requests               # to get the webpage
from bs4 import BeautifulSoup # to parse the webpage

import pandas as pd
import re                     # regular expression operators

from datetime import datetime

required_city = "Mumbai"
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
   'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
   's': required_city,
   'stack': 'aws',
   'locale': 'en',
   'filter': 'international',
   'place-types': 'settlement,airport,district',
   'order': 'importance',
   'a': 'true',
   'format': 'json'
})

result = requests.get(location_url).json()
location_code = str(result['response']['results']['results'][0]['id'])
print(location_code)


response = requests.get('https://www.bbc.com/weather/' + location_code)
soup = BeautifulSoup(response.content,'html.parser') 
daily_high_values = soup.find_all('span', attrs={'class': 'wr-day-temperature__high-value'})
daily_high_values_list = [high_value.text.strip().split()[0].replace('°','') for high_value in daily_high_values]
daily_low_values  = soup.find_all('span', attrs={'class': 'wr-day-temperature__low-value'})
daily_low_values_list = [low_value.text.strip().split()[0].replace('°','') for low_value in daily_low_values]
daily_summary = soup.find('div', attrs={'class': 'wr-day-summary'})
daily_summary_list = re.findall('[a-zA-Z][^A-Z]*', daily_summary.text) #split the string on uppercase

from datetime import timedelta

datelist = [(datetime.today()+timedelta(i)).strftime('%y-%m-%d') for i in range(len(daily_low_values))]
zipped = zip(datelist, daily_high_values_list, daily_low_values_list, daily_summary_list)
df = pd.DataFrame(list(zipped), columns=['Date', 'High','Low', 'Summary'])
display(df)

1275339


,Date,High,Low,Summary
0,22-06-25,30,24,Light rain showers and a gentle breeze
1,22-06-26,30,26,Light cloud and a gentle breeze
2,22-06-27,30,26,Thundery showers and a moderate breeze
3,22-06-28,30,26,Thundery showers and a gentle breeze
4,22-06-29,29,26,Thundery showers and a gentle breeze
5,22-06-30,28,26,Thundery showers and a moderate breeze
6,22-07-01,29,26,Thundery showers and a moderate breeze
7,22-07-02,30,26,Thundery showers and a moderate breeze
8,22-07-03,29,26,Thundery showers and a moderate breeze
9,22-07-04,29,26,Thundery showers and a moderate breeze


In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

dic = {}
urls = ['https://www.imdb.com/search/title/?title_type=feature&year=2012-03-25,2012-03-31&sort=runtime,asc', 'https://www.imdb.com/search/title/?title_type=feature&year=2012-03-25,2012-03-31&sort=runtime,asc&start=51&ref_=adv_nxt', 'https://www.imdb.com/search/title/?title_type=feature&year=2012-03-25,2012-03-31&sort=runtime,asc&start=101&ref_=adv_nxt', 'https://www.imdb.com/search/title/?title_type=feature&year=2012-03-25,2012-03-31&sort=runtime,asc&start=151&ref_=adv_nxt']
url = ['https://www.imdb.com/search/title/?title_type=feature&year=2012-03-25,2012-03-31&sort=runtime,asc']
for i in url:
  html = requests.get(i).content
  soup = bs(html)
  genre = soup.find_all('span', attrs={'class': 'genre'})
  print(genre.get_text())

AttributeError: ignored

## Wikimedia

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia as wk
wk.search('IIT Madras') #results=2
wk.summary('IIT Madras') #sentences=2
full_page = wk.page('IIT Madras')
print('-----')
#full_page.content
#print(full_page.url)
#print(full_page.references)
#print(full_page.title)
#print(full_page.images)
html = wk.page('IIT Madras').html().encode('UTF-8')
import pandas as pd
df = pd.read_html(html)[5]
print(df)

-----
                     Fest name                               Department
0                    Aero fest                    Aerospace Engineering
1                     ElecFest                   Electrical Engineering
2                     Samanvay         Department of Management Studies
3                      Biofest                            Biotechnology
4                       ExeBit         Computer Science and Engineering
5                      Amalgam  Metallurgical and Materials Engineering
6                     CEA Fest                        Civil Engineering
7                     Chemplus                     Chemical Engineering
8                       Forays                              Mathematics
9                    Mechanica                   Mechanical Engineering
10                       Wavez                        Ocean Engineering
11  Annual Academic Conference           Humanities and Social Sciences
12                    Bhoutics                            

## PDFs !!!

In [ ]:
!pip install tabula==1.0.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tabula: filename=tabula-1.0.4-py3-none-any.whl size=10610 sha256=5f5efbd6490579c1ddd4de03d35fa85016cea0350cf9a2f2bf4d4a4475f90835
  Stored in directory: /root/.cache/pip/wheels/04/80/72/97cd0a10f499c86ca59e526027b558a02a968287f0ca982baa
Successfully built tabula
  Attempting uninstall: tabula
    Found existing installation: tabula 1.0.5
    Uninstalling tabula-1.0.5:
      Successfully uninstalled tabula-1.0.5


In [ ]:
import os
import requests
import urllib.request
import pandas as pd
import tabula #to parse the pdf files
from urllib.parse import urljoin
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
url = "https://www.premierleague.com/publications"
folder_location = r'/content/drive/MyDrive/Colab Notebooks/premier_league'
# Create a folder to store the downloaded pdfs
if not os.path.exists(folder_location):
    os.mkdir(folder_location)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
 


In [ ]:

for each in soup.find_all("a",  attrs={'href': re.compile('.*\.pdf')}):
    filename = os.path.join("premier_league", each['href'].split('/')[-1])
    print(filename)
    requests.get(urljoin(url,each['href'])).content

premier_league/PL_Handbook_2021_22_DIGITAL_07-04-22.pdf
premier_league/PL_2021-22_Youth_Development_Rules_23.09.pdf
premier_league/PL2_2021-22_Handbook_23.09.pdf
premier_league/PDL_2021-22_Handbook_23.09.pdf
premier_league/U18-PL_2021-22_Handbook_23.09.pdf
premier_league/U18PDL_2022-21_Handbook_23.09.pdf
premier_league/FP-Player-Parent-Handbook-2020-21.pdf
premier_league/YDP-Player-Parent-Handbook-2020-21.pdf
premier_league/Professional-Retain-List-2021-22.pdf
premier_league/Professional-Retain-List-2021-22.pdf
premier_league/PL-Commitment-re.-abusive-and-discriminatory-conduct-August-2021-.pdf
premier_league/Accessible-Stadia-Guidelines-Premier-League-Report-2017-appendix.pdf


KeyboardInterrupt: ignored

In [ ]:
from tabula import convert_into
import tabula

combined_pdf = folder_location + "/This-is-PL-Interactive-Combined.pdf"
convert_into(combined_pdf, folder_location +"/table_output.csv", output_format="csv",pages = 18,area=[[275,504,640,900]])
pd.read_csv(folder_location+"/table_output.csv", encoding="ANSI")